#### Scott Wilcox
#### A01810798
#### FIN 5350
#### Homework 2

### Binomial Option Pricing Model

In [55]:
#import numpy
import numpy as np
from scipy.stats import binom

In [56]:
#Payoff functions
def callPayoff(spot,strike):
    return np.maximum(spot-strike,0.0)

def putPayoff(spot,strike):
    return np.maximum(strike-spot,0.0)

In [57]:
#European Binomial Option Pricer

def eurBinPricer(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes=n+1
    h=T/n
    u=np.exp((r-q)*h+v*np.sqrt(h))
    d=np.exp((r-q)*h-v*np.sqrt(h))
    pu=(np.exp((r-q)*h)-d)/(u-d)
    pd=1.0-pu
    disc=np.exp(-r*h)
    
    Ct=np.empty(nodes)
    St=np.empty(nodes)
    
    for i in range(nodes):
        St[i]=S*(u**(n-i))*(d**i)
        Ct[i]=payoff(St[i], K)
    
    for t in range((n-1),-1,-1):
        for j in range(t+1):
            Ct[j]=disc*(pu*Ct[j]+pd*Ct[j+1])
    
    return Ct[0]

In [23]:
## European Pricer with Delta, B, Premium

def eurGiven(S, K, r, q, T, n, u, d, payoff, verbose = True):
    nodes=n+1
    h=T/n
    pu=(np.exp((r-q)*h)-d)/(u-d)
    pd=1.0-pu
    disc=np.exp(-r*h)
    
    Ct=np.zeros((nodes,n+1))
    St=np.zeros((nodes,n+1))
    Dt=np.zeros((nodes,n+1))
    Bt=np.zeros((nodes,n+1))
    
    for i in range(nodes):
        St[i,n]=S*(u**(n-i))*(d**i)
        Ct[i,n]=payoff(St[i,n],K)    

    for t in range((n-1),-1,-1):
        for j in range(t+1):
            St[j,t]=St[j,t+1]/u
            Ct[j,t]=disc*((pu*Ct[j,t+1])+(pd*Ct[j+1,t+1]))
            Dt[j,t]=np.exp(-q*h)*(Ct[j,t+1]-Ct[j+1,t+1])/(St[j,t]*(u-d))
            Bt[j,t]=Ct[j,t]-(Dt[j,t]*St[j,t])
    
    return Dt,Bt,Ct

In [25]:
#American Binomial Option Pricer

def amerBinPricer(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes=n+1
    h=T/n
    u=np.exp((r-q)*h+v*np.sqrt(h))
    d=np.exp((r-q)*h-v*np.sqrt(h))
    pu=(np.exp((r-q)*h)-d)/(u-d)
    pd=1.0-pu
    disc=np.exp(-r*h)
    
    Ct=np.empty(nodes)
    St=np.empty(nodes)
    
    for i in range(nodes):
        St[i]=S*(u**(n-i))*(d**i)
        Ct[i]=payoff(St[i],K)
    
    for t in range((n-1),-1,-1):
        for j in range(t):
            Ct[j]=disc*(pu*Ct[j]+pd*Ct[j+1])
            St[j]=St[j]/u
            earlyPay=K-St[j]
            Ct[j]=np.maximum(Ct[j],earlyPay)
    
    return Ct[0]

#### Problem 1
##### S=100, K=105, r=0.08, T=0.5, q=0, n=1, u=1.3, d=0.8

In [15]:
eurGiven(100,105,0.08,0,0.5,1,1.3,0.8,callPayoff,verbose=False)

(array([[0.50, 0.00],
        [0.00, 0.00]]), array([[-38.43, 0.00],
        [0.00, 0.00]]), array([[11.57, 25.00],
        [0.00, 0.00]]))

Delta = 0.5; B = -38.43; Premium = 11.57

In [14]:
eurGiven(100,105,0.08,0,0.5,1,1.3,0.8,putPayoff,verbose=False)

(array([[-0.50, 0.00],
        [0.00, 0.00]]), array([[62.45, 0.00],
        [0.00, 0.00]]), array([[12.45, 0.00],
        [0.00, 25.00]]))

Delta = -0.5; B = 62.45; Premium = 12.45

#### Problem 2
##### S=100, K=95, r=0.08, T=0.5, q=0, n=1, u=1.3, d=0.8

In [18]:
eurGiven(100,95,0.08,0,0.5,1,1.3,0.8,putPayoff,verbose=False)

(array([[-0.30, 0.00],
        [0.00, 0.00]]), array([[37.47, 0.00],
        [0.00, 0.00]]), array([[7.47, 0.00],
        [0.00, 15.00]]))

The price of the European put is $7.471.

In [19]:
eurGiven(100,95,0.08,0,0.5,1,1.3,0.8,callPayoff,verbose=False)

(array([[0.70, 0.00],
        [0.00, 0.00]]), array([[-53.80, 0.00],
        [0.00, 0.00]]), array([[16.20, 35.00],
        [0.00, 0.00]]))

The fair price of the European call is $16.196.

At 17, the option is overvalued. The arbitrage includes borrowing 53.804 and buying 0.7 shares. Sell a call option. Arbitrage profit is 17-16.196=0.804.

At 15.5, the option is undervalued. The arbitrage includes buying a call option. Lend 53.804 and sell 0.7 shares. Arbitrage profit is 16.196-15.5=0.696.

#### Problem 3
##### S=100, K=95, r=0.08, T=1, q=0, u=1.3, d=0.8, n=2

In [26]:
eurGiven(100,95,0.08,0,1,2,1.3,0.8,callPayoff,verbose=False)

(array([[0.69, 1.00, 0.00],
        [0.00, 0.23, 0.00],
        [0.00, 0.00, 0.00]]), array([[-49.13, -91.27, 0.00],
        [0.00, -13.84, 0.00],
        [0.00, 0.00, 0.00]]), array([[19.99, 38.73, 74.00],
        [0.00, 4.16, 9.00],
        [0.00, 0.00, 0.00]]))

Let Su indicate the spot price node after two up movements, Sd after two down movements.

Su has Delta = 1.0; B = -91.27; Premium = 38.73

Sd has Delta = 0.225; B = -13.84; Premium = 4.16

S0 has Delta = 0.69; B = -49.13; Premium = 19.99

#### Problem 4
##### Repeat the option price calculation in the previous question for stock prices of 80, 90, 110, 120, and 130 with n=3. What happens to the initial option delta as the stock price increases?

In [29]:
eurGiven(80,95,0.08,0,1,2,1.3,0.8,callPayoff,verbose=False)

(array([[0.47, 0.77, 0.00],
        [0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00]]), array([[-28.60, -61.80, 0.00],
        [0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00]]), array([[8.61, 18.60, 40.20],
        [0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00]]))

In [30]:
eurGiven(90,95,0.08,0,1,2,1.3,0.8,callPayoff,verbose=False)

(array([[0.59, 0.98, 0.00],
        [0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00]]), array([[-40.62, -87.78, 0.00],
        [0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00]]), array([[12.23, 26.42, 57.10],
        [0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00]]))

In [32]:
eurGiven(110,95,0.08,0,1,2,1.3,0.8,callPayoff,verbose=False)

(array([[0.78, 1.00, 0.00],
        [0.00, 0.44, 0.00],
        [0.00, 0.00, 0.00]]), array([[-57.09, -91.27, 0.00],
        [0.00, -29.82, 0.00],
        [0.00, 0.00, 0.00]]), array([[28.41, 51.73, 90.90],
        [0.00, 8.98, 19.40],
        [0.00, 0.00, 0.00]]))

In [33]:
eurGiven(120,95,0.08,0,1,2,1.3,0.8,callPayoff,verbose=False)

(array([[0.85, 1.00, 0.00],
        [0.00, 0.62, 0.00],
        [0.00, 0.00, 0.00]]), array([[-65.05, -91.27, 0.00],
        [0.00, -45.81, 0.00],
        [0.00, 0.00, 0.00]]), array([[36.82, 64.73, 107.80],
        [0.00, 13.79, 29.80],
        [0.00, 0.00, 0.00]]))

In [34]:
eurGiven(130,95,0.08,0,1,2,1.3,0.8,callPayoff,verbose=False)

(array([[0.91, 1.00, 0.00],
        [0.00, 0.77, 0.00],
        [0.00, 0.00, 0.00]]), array([[-73.01, -91.27, 0.00],
        [0.00, -61.80, 0.00],
        [0.00, 0.00, 0.00]]), array([[45.23, 77.73, 124.70],
        [0.00, 18.60, 40.20],
        [0.00, 0.00, 0.00]]))

The initial delta option is converging to 1. It begins increasing toward 1 at a fast rate and then begins to slow as the spot price surpasses the strike price. As the stock price increases for a call option, the option is more likely to be exercised, meaning the option will simply be held to maturity.

#### Problem 5
##### S=100, K=95, r=0.08, v=0.30, q=0, T=1, n=3

In [65]:
eurBinPricer(100,95,0.08,0.3,0,1,3,callPayoff,verbose=False)

18.28255220737056

The premium for an American call option is 18.2826. There is no early exercise. The American call option premium is the same as the European call option premium since dividends are zero.

In [68]:
print("Premium for the European put is ",eurBinPricer(100,95,0.08,0.3,0,1,3,putPayoff,verbose=False))

Premium for the European put is  5.97860511410097


In [70]:
#Put-call parity: S+p0=c0+(K/(1+r)^t)
print(5.978605-18.2826)
print((95/((1+0.08)**1))-100)

-12.303994999999999
-12.037037037037038


Values differ by about 0.3, or 30 percentage points. Put-call parity appears to hold.

#### Problem 6
##### S=40, K=40, r=0.08, v=0.3, q=0, T=0.5, n=3

In [54]:
print("u = ", np.exp((0.08 - 0) * (0.5/3) + 0.3 * np.sqrt(0.5/3)))
print("d = ", np.exp((0.08 - 0) * (0.5/3) - 0.3 * np.sqrt(0.5/3)))

u =  1.1454617381452392
d =  0.8966038495199921


In [75]:
print("Premium for the European call is ",eurBinPricer(40,40,0.08,0.3,0,0.5,3,callPayoff,verbose=False))

Premium for the European call is  4.377429513517246


In [76]:
print("Premium for the European put is ",eurBinPricer(40,40,0.08,0.3,0,0.5,3,putPayoff,verbose=False))

Premium for the European put is  2.8090070796101805


In [79]:
print("Premium for the American put is ",amerBinPricer(40,40,0.08,0.3,0,0.5,3,putPayoff,verbose=False))

Premium for the American put is  0.8679901896366252


The premium for the American call is 4.3774, the same as the European call since dividends are zero.